# Load in Data and Required Packages

In [2]:
import dash
from jupyter_dash import JupyterDash
import dash_core_components as dcc
import dash_html_components as html
import dash_table as dt
from dash.dependencies import Input, Output, State
import inspect
import pandas as pd
import numpy as np
import warnings
import dash_bootstrap_components as dbc
import dash_table_experiments as dt
from dash.exceptions import PreventUpdate
warnings.filterwarnings('ignore')
import matplotlib.pyplot as plt
import json

In [5]:
import json

data_path = 'questions.json'

# Open and read the JSON file
with open(data_path, 'r') as json_file:
    data = json.load(json_file)
# Create a DataFrame from the JSON data
df = pd.DataFrame(data)
df

,question,solution,answer
0,"The Culinary Institute of America, the best cu...",def solution():\n #40 people paid $3000 for...,720000.000000
1,Melanie has $45. She shops at a grocery store ...,def solution():\n #Melanie started with $45...,0.800000
2,Dianne is Dunkin' Brands CFO. She has to cut 5...,def solution():\n #Cut 5% of the budget for...,4.940000
3,"The scone cost $4, the muffin cost $3, the don...","def solution():\n #The scone cost $4, the m...",24.000000
4,4 pizzas cost $32. 2 pizzas cost $16. How much...,def solution():\n # 4 pizzas cost $32\n ...,8.000000
5,Gina is in a candy store that sells 10 gumball...,def solution():\n #Gina has 5 dollars\n ...,0.500000
6,"For my 18th birthday, my parents gave me $100....",def solution():\n #I got $100 for my 18th b...,1459.374246
7,Chase and his parents are going on a trip to t...,def solution():\n #Chase and his parents ar...,275.000000


# Build Dash App

In [6]:
markdown_text = '''
Please read the following two questions carefully. Select the question you think is better. Make a decision based on:
a) the correctness of the solution (you may use a calculator or internet to verify the solution) 
b) the correctness of the reasoning contained within the Python code that executes the solution
c) the appropriateness of the question (i.e., is it a strange question?, is it appropriate for a K-12 student?)

After selecting the question you think is best, please answer the follow-up questions regarding the grade level, mathematical operations present in the question, and general topic the question is about.

If both questions are equally good based on the criteria above, you may select the 'Tie- both good' option. If both questions are equally bad based on the criteria above, you may select the 'Tie- both bad' option. Please use these options sparingly and try to declare a clear winner whenever possible. 

If the question/reasoning is nonsensical or highly ungrammatical/confusing, label it with "F" (for fail) rather than selecting a winner. 
'''

In [7]:
external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css']

In [89]:
app = JupyterDash(__name__, external_stylesheets=external_stylesheets)

app.layout = html.Div(
    [
        html.H1("Math Question Generator Feedback Portal"),    
        
        dcc.Markdown(children = markdown_text),
        
        html.Div([
            
            html.Button(id='submit', n_clicks=0, children = 'Display Questions', style = {'fontsize': '12'}),
            dcc.Store(id='button-clicked', data=0)

        ], style={'width': '12%', 'float': 'left'}),
        
        html.Div([
            html.H2(id = 'question1_label'),
            dcc.Markdown(id='question1'),
            dcc.Markdown(id= 'question1_solution'),
            dcc.Markdown(id = 'question1_answer'),
        ], style={'width': '42%', 'float': 'left', 'font-weight': 'bold', 'font-size': '14px', 'white-space': 'pre-wrap', 'display': 'inline-block'}), 
        
        html.Div([
            html.H2(id = 'question2_label'),
            dcc.Markdown(id='question2'),
            dcc.Markdown(id= 'question2_solution'),
            dcc.Markdown(id = 'question2_answer'),
        ], style={'width': '42%', 'justify-content': 'space-between', 'font-weight': 'bold', 'font-size': '12px', 'display': 'inline-block', 'white-space': 'pre-wrap'}),
        
        html.Div(
            [
                html.Label('Select the best option:', style={'font-weight': 'bold'}),
                dcc.RadioItems(
                    id='selected_question',
                    options=[
                        {'label': 'Question 1', 'value': 'Question 1'},
                        {'label': 'Question 2', 'value': 'Question 2'},
                        {'label': 'Tie- both good', 'value': 'Tie- both good'},
                        {'label': 'Tie- both bad', 'value': 'Tie- both bad'}
                    ]
                ),
                html.Div(html.Button('Submit Feedback', id='submit-val', n_clicks=0))
            ],
            style={'width': '100%', 'display': 'none'},
            id='standard-container')
    ]
)

@app.callback([Output(component_id="question1",component_property="children"), 
               Output(component_id="question1_solution",component_property="children"), 
              Output(component_id = 'question1_answer', component_property = 'children'), 
               Output(component_id = 'question1_label', component_property = 'children'),
              Output(component_id="question2",component_property="children"), 
               Output(component_id="question2_solution",component_property="children"), 
              Output(component_id = 'question2_answer', component_property = 'children'),
              Output(component_id = 'question2_label', component_property = 'children')],
                  [Input(component_id = 'submit', component_property= 'n_clicks')])

def pull_questions(n_clicks, df=df):
    if n_clicks>0:
        df_temp = df.sample(n = 2, replace= False)
        question1 = df_temp.iloc[0]['question']
        question1 = "Question: \n" + question1
        question1_solution = df_temp.iloc[0]['solution']
        question1_solution = "Solution: \n" + question1_solution
        question1_answer = "Answer: \n" + str(df_temp.iloc[0]['answer'])
        question1_label = 'Question 1'
        
        question2 = df_temp.iloc[1]['question']
        question2 = "Question: \n" + question2
        question2_solution = df_temp.iloc[1]['solution']
        question2_solution = "Solution: \n" + question2_solution
        question2_answer = "Answer: \n" + str(df_temp.iloc[1]['answer'])
        question2_label = "Question 2"
        return question1, question1_solution, question1_answer, question1_label, question2, question2_solution, question2_answer, question2_label
    if n_clicks==0:
        return '', '', '', '', '', '', '', ''

@app.callback(
    Output('standard-container', 'style'),
    [Input('submit', 'n_clicks')],
    State('button-clicked', 'data')
)
def show_hide_radioitems(n_clicks, button_clicked):
    if n_clicks > button_clicked:
        return {'display': 'block'}
    else:
        return {'display': 'none'}



if __name__== "__main__":
    app.run_server(mode= 'external', host = "0.0.0.0", debug=True)

Dash app running on http://0.0.0.0:8050/


In [49]:
df

,question,solution,answer
0,"The Culinary Institute of America, the best cu...",def solution():\n #40 people paid $3000 for...,720000.000000
1,Melanie has $45. She shops at a grocery store ...,def solution():\n #Melanie started with $45...,0.800000
2,Dianne is Dunkin' Brands CFO. She has to cut 5...,def solution():\n #Cut 5% of the budget for...,4.940000
3,"The scone cost $4, the muffin cost $3, the don...","def solution():\n #The scone cost $4, the m...",24.000000
4,4 pizzas cost $32. 2 pizzas cost $16. How much...,def solution():\n # 4 pizzas cost $32\n ...,8.000000
5,Gina is in a candy store that sells 10 gumball...,def solution():\n #Gina has 5 dollars\n ...,0.500000
6,"For my 18th birthday, my parents gave me $100....",def solution():\n #I got $100 for my 18th b...,1459.374246
7,Chase and his parents are going on a trip to t...,def solution():\n #Chase and his parents ar...,275.000000
